In [1]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
# from vgg16 import VGG16_LSTM
import pickle
import torch.nn as nn
import torch
import datetime
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN
import logging
import torch
import torch.nn as nn
from vivit import ViViTBackbone
import gc
from einops import rearrange
from tqdm import tqdm
batchsz = 4
frame_nums=15
start_epoch = 0
test_flage=False
model_name ="15framesfullshot"
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s]::%(module)s::%(levelname)s::%(message)s')
streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
fileHandler = logging.FileHandler('./LOG/personalityLog.log')
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)
USE_CUDA = torch.cuda.is_available()

/tmp/ipykernel_3841102/3498005901.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
logger.debug('============={}+Video Vision Transformer Strat!============='.format(model_name))
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:1' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:', device)

[2024-09-19 14:32:26,283]::3493104939::DEBUG::=============15framesfullshot+Video Vision Transformer Strat!=============


True
학습을 진행하는 기기: cuda:1


In [3]:
def getdata(pathname,datatype):
    if datatype=="train":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            final_data_set.extend(pickle.load(output_file))
        return final_data_set
    elif datatype=="valid":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            final_data_set.extend(pickle.load(output_file))
        return final_data_set
print(1)

1


In [4]:

save_model_file_path = './save_swintransformer_folder/{}_{}.{}'
print(1)

1


In [5]:
# validation_set_data=getdata("/data/qx/secwayofKrData/15Frames/test_set.dat","valid")
# print(1)
validation_set_data=getdata("/data/qx/firstwayofKrData/15FramesDatasetKr/test_set.dat","valid")
print(1)

1


In [6]:
# train_set_data=getdata("/data/qx/secwayofKrData/15Frames/train_set.dat","train")
# print(1)
train_set_data=getdata("/data/qx/firstwayofKrData/15FramesDatasetKr/train_set.dat","train")
print(1)

1


In [7]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])
print(1)

1


In [8]:
class ChalearnDataset(Dataset):
    def __init__(self,fullshot,tagdata,transform=None):
        self.fullshot=fullshot
        self.tagdata=tagdata
        self.transform = transform  # 표준화 여부
    def __len__(self):
        return len(self.fullshot)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()#텐서의 경우 목록으로 돌아가기
        fullshot=self.fullshot[idx]
        fullshot=torch.FloatTensor(fullshot)
        big_five_sorces=self.tagdata[idx]
        big_five_sorces = torch.FloatTensor(big_five_sorces)
        return fullshot,big_five_sorces
print(1)

1


In [9]:
num_workerssz = 20
lr = 3e-05
epochs = 120
print(1)

1


In [10]:
train_set_data = ChalearnDataset(fullshot = train_set_data[1],tagdata=train_set_data[3],transform=transform)
val_set_data = ChalearnDataset(fullshot = validation_set_data[1],tagdata=validation_set_data[3],transform=transform)
train_dataloader = DataLoader(dataset=train_set_data, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
val_dataloader = DataLoader(dataset=val_set_data, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
max_value=0
print(1)

1


In [11]:
model=ViViTBackbone(2, 14, 14, 2, 2, 2, 5, 96, 6, 10, 8, model=3)#num_frames=frame_nums,batchsz = batchsz
model=model.to(device)
criterion = torch.nn.L1Loss().to(device) # 손실함수
optimizer = torch.optim.AdamW(model.parameters(), lr=lr) # 옵티마이저

if test_flage==True:
    checkpoint=torch.load(save_model_file_path.format('model',start_epoch,'pth'), map_location=device)
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    criterion.load_state_dict(checkpoint["loss"])
train_avg_loss0=[]
val_avg_loss0=[]
print(1)

1


In [12]:
from torchinfo import summary

summary(model, input_size = ((4, 3,15, 224, 224)), col_names = ['input_size','output_size','num_params'],verbose = 1)

Layer (type:depth-idx)                             Input Shape               Output Shape              Param #
ViViTBackbone                                      [4, 3, 15, 224, 224]      [4, 5]                    3,146,016
├─VideoResNet: 1-1                                 [4, 3, 15, 224, 224]      [4, 512, 2, 14, 14]       205,200
│    └─R2Plus1dStem: 2-1                           [4, 3, 15, 224, 224]      [4, 64, 15, 112, 112]     --
│    │    └─Conv3d: 3-1                            [4, 3, 15, 224, 224]      [4, 45, 15, 112, 112]     6,615
│    │    └─BatchNorm3d: 3-2                       [4, 45, 15, 112, 112]     [4, 45, 15, 112, 112]     90
│    │    └─ReLU: 3-3                              [4, 45, 15, 112, 112]     [4, 45, 15, 112, 112]     --
│    │    └─Conv3d: 3-4                            [4, 45, 15, 112, 112]     [4, 64, 15, 112, 112]     8,640
│    │    └─BatchNorm3d: 3-5                       [4, 64, 15, 112, 112]     [4, 64, 15, 112, 112]     128
│    │    └─ReLU: 3-6 

Layer (type:depth-idx)                             Input Shape               Output Shape              Param #
ViViTBackbone                                      [4, 3, 15, 224, 224]      [4, 5]                    3,146,016
├─VideoResNet: 1-1                                 [4, 3, 15, 224, 224]      [4, 512, 2, 14, 14]       205,200
│    └─R2Plus1dStem: 2-1                           [4, 3, 15, 224, 224]      [4, 64, 15, 112, 112]     --
│    │    └─Conv3d: 3-1                            [4, 3, 15, 224, 224]      [4, 45, 15, 112, 112]     6,615
│    │    └─BatchNorm3d: 3-2                       [4, 45, 15, 112, 112]     [4, 45, 15, 112, 112]     90
│    │    └─ReLU: 3-3                              [4, 45, 15, 112, 112]     [4, 45, 15, 112, 112]     --
│    │    └─Conv3d: 3-4                            [4, 45, 15, 112, 112]     [4, 64, 15, 112, 112]     8,640
│    │    └─BatchNorm3d: 3-5                       [4, 64, 15, 112, 112]     [4, 64, 15, 112, 112]     128
│    │    └─ReLU: 3-6 

In [13]:
def extract_ocean(data,batch_size):
    o=[]
    c=[]
    e=[]
    a=[]
    n=[]
    for i in range(batch_size):
        o.append(data[i][0].item())
        c.append(data[i][1].item())
        e.append(data[i][2].item())
        a.append(data[i][3].item())
        n.append(data[i][4].item())
    o = torch.tensor(o) 
    c = torch.tensor(c) 
    e = torch.tensor(e) 
    a = torch.tensor(a) 
    n = torch.tensor(n) 
    return o,c,e,a,n

In [14]:

max_value = float('inf')
for i in range(start_epoch, epochs):
    train_avg_loss = 0
    val_avg_loss = 0
    train_avg_o_loss = 0
    train_avg_c_loss = 0
    train_avg_e_loss = 0
    train_avg_a_loss = 0
    train_avg_n_loss = 0
    val_avg_o_loss = 0
    val_avg_c_loss = 0
    val_avg_e_loss = 0
    val_avg_a_loss = 0
    val_avg_n_loss = 0
    for fullshot,big_five_data in tqdm(train_dataloader):
        fullshot=fullshot.permute(0,4,1,2,3)
        big_five_data=big_five_data.permute(0,2,1)
        big_five_data=big_five_data.squeeze()
        fullshot = fullshot.to(device)
        big_five_data = big_five_data.to(device)
        optimizer.zero_grad()  # 기울기가 0이 됩니다.
        hypothesis = model(fullshot) # 모델의 예측 결과를 저장합니다.
        loss = criterion(hypothesis, big_five_data)  # 예측된 결과와 실제 태그 사이의 손실 값을 저장합니다.
        loss.backward()  # 역방향 전파입니다. 
        optimizer.step()  
        train_avg_loss += loss  
    train_avg_loss=train_avg_loss/len(train_dataloader)
    with torch.no_grad():#validate
        for fullshot,big_five_data in tqdm(val_dataloader):
            fullshot=fullshot.permute(0,4,1,2,3)
            big_five_data=big_five_data.permute(0,2,1)
            big_five_data=big_five_data.squeeze()
            fullshot = fullshot.to(device)
            big_five_data = big_five_data.to(device)
            hypothesis = model(fullshot)
            val_loss = criterion(hypothesis, big_five_data)
            hypothesiso,hypothesisc,hypothesise,hypothesisa,hypothesisn=extract_ocean(hypothesis,batchsz)
            big_five_datao,big_five_datac,big_five_datae,big_five_dataa,big_five_datan=extract_ocean(big_five_data,batchsz)
            val_o_loss = criterion(hypothesiso, big_five_datao) 
            val_c_loss = criterion(hypothesisc, big_five_datac) 
            val_e_loss = criterion(hypothesise, big_five_datae) 
            val_a_loss = criterion(hypothesisa, big_five_dataa) 
            val_n_loss = criterion(hypothesisn, big_five_datan)  
            val_avg_loss += val_loss
            val_avg_o_loss += val_o_loss
            val_avg_c_loss += val_c_loss
            val_avg_e_loss += val_e_loss
            val_avg_a_loss += val_a_loss
            val_avg_n_loss += val_n_loss
        val_avg_loss=val_avg_loss/len(val_dataloader)
        val_avg_o_loss=val_avg_o_loss/len(val_dataloader)
        val_avg_c_loss=val_avg_c_loss/len(val_dataloader)
        val_avg_e_loss=val_avg_e_loss/len(val_dataloader)
        val_avg_a_loss=val_avg_a_loss/len(val_dataloader)
        val_avg_n_loss=val_avg_n_loss/len(val_dataloader)
    torch.cuda.empty_cache()
    start_epoch+=1
    if val_avg_loss < max_value:
        max_value=val_avg_loss
        torch.save({
            'epoch': i+1,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'loss': val_avg_loss,
        }, save_model_file_path.format('model',start_epoch,'pth'))
    logger.debug('Epoch: {} , 1-MAE: {:.4f} , 1-Training Loss: {:.4f} , 1-val_avg_o_loss:{:.4f} , 1-val_avg_c_loss:{:.4f} , 1-val_avg_e_loss:{:.4f} , 1-val_avg_a_loss:{:.4f} , 1-val_avg_n_loss:{:.4f}'.format(i+1, 1-val_avg_loss, 1-train_avg_loss,1-val_avg_o_loss, 1-val_avg_c_loss, 1-val_avg_e_loss, 1-val_avg_a_loss,1-val_avg_n_loss))


100%|██████████| 193/193 [00:17<00:00, 11.00it/s]
[2024-09-19 14:35:48,055]::3746580590::DEBUG::Epoch: 1 , 1-MAE: 0.8647 , 1-Training Loss: 0.9331 , 1-val_avg_o_loss:0.8710 , 1-val_avg_c_loss:0.8620 , 1-val_avg_e_loss:0.8536 , 1-val_avg_a_loss:0.8938 , 1-val_avg_n_loss:0.8430
100%|██████████| 193/193 [00:17<00:00, 10.93it/s]
[2024-09-19 14:38:59,357]::3746580590::DEBUG::Epoch: 2 , 1-MAE: 0.8586 , 1-Training Loss: 0.9637 , 1-val_avg_o_loss:0.8672 , 1-val_avg_c_loss:0.8683 , 1-val_avg_e_loss:0.8506 , 1-val_avg_a_loss:0.8831 , 1-val_avg_n_loss:0.8237
100%|██████████| 193/193 [00:17<00:00, 10.92it/s]
[2024-09-19 14:42:10,915]::3746580590::DEBUG::Epoch: 3 , 1-MAE: 0.8595 , 1-Training Loss: 0.9749 , 1-val_avg_o_loss:0.8694 , 1-val_avg_c_loss:0.8727 , 1-val_avg_e_loss:0.8490 , 1-val_avg_a_loss:0.8891 , 1-val_avg_n_loss:0.8173
100%|██████████| 193/193 [00:17<00:00, 10.86it/s]
[2024-09-19 14:45:22,742]::3746580590::DEBUG::Epoch: 4 , 1-MAE: 0.8601 , 1-Training Loss: 0.9784 , 1-val_avg_o_loss:0.8